# Titanic Survival Prediction
This notebook uses XGBoost to predict survival on the Titanic dataset, with feature engineering and preprocessing pipelines.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

## Load and preprocess data

In [4]:
# Load Titanic dataset
data = pd.read_csv("titanic/train.csv")

# Feature Engineering
data["FamilySize"] = data["SibSp"] + data["Parch"]
data["IsAlone"] = (data["FamilySize"] == 0).astype(int)
data["Title"] = data["Name"].str.extract(" ([A-Za-z]+)\\.", expand=False)
data["Title"] = data["Title"].replace(
    ['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 
     'Sir', 'Jonkheer', 'Dona'], 'Rare')
data["Title"] = data["Title"].replace(['Mlle', 'Ms'], 'Miss')
data["Title"] = data["Title"].replace('Mme', 'Mrs')

# Drop columns not needed
data.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1, inplace=True)

## Split data

In [5]:
# Split features and target
y = data["Survived"]
X = data.drop("Survived", axis=1)

# Split into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## Build preprocessing pipeline

In [6]:
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
numerical_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]

# Define transformers
numerical_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

## Define and train model

In [7]:
# Create full pipeline with XGBoost
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(n_estimators=100, learning_rate=0.1,
                                 use_label_encoder=False, eval_metric='logloss',
                                 random_state=42))
])

# Train model
model.fit(X_train, y_train)

c:\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:03:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare',
                                                   'FamilySize']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('classifier',
                 XGBClassifier(base_score=None, booster...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None, ...))])

## Evaluate accuracy

In [8]:
# Predict and evaluate
predictions = model.predict(X_valid)
accuracy = accuracy_score(y_valid, predictions)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.8268156424581006


## Cross-validation

In [9]:
cv_scores = cross_val_score(model, X, y, cv=5)
print("Cross-validated Accuracy:", cv_scores.mean())

c:\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:03:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:03:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:03:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:03:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Python312\Lib\sit

Cross-validated Accuracy: 0.8350197727700708
